In [12]:
import open3d as o3d
import numpy as np
import math
import glob, os, re
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import itertools
from pyvis.network import Network

In [13]:
# 対象フォルダのパスを指定
folder_path = "/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/2"
#'/home/aichi2204/Documents/bkl2go/20240412-library2/aichi-20240412-library2_croped_mini_1/tmp'  # ここを対象フォルダのパスに置き換えてください
# パターンにマッチするファイルをすべて取得
file_pattern = os.path.join(folder_path, '[0-9]*.pcd')
file_paths = glob.glob(file_pattern)

In [14]:
pattern = r"/(\d+)\.pcd"

pcd = o3d.io.read_point_cloud(file_paths[0])
l = int(re.search(pattern, file_paths[0]).group(1))
pcd_label = [l] * len(pcd.points)
# pcd_idx = np.array([i for i in range(len(pcd.points))])

for i in range(1, len(file_paths)):
    pcd_tmp  = o3d.io.read_point_cloud(file_paths[i])
    l = int(re.search(pattern, file_paths[i]).group(1))

    pcd += pcd_tmp
    pcd_label += [l] * len(pcd_tmp.points)

pcd_label = np.array(pcd_label)
pcd_idx = np.array([i for i in range(len(pcd.points))])

In [15]:
pcd_noise = o3d.io.read_point_cloud( os.path.join(folder_path, 'noise.pcd'))
pcd_noise_label = np.full(len(pcd_noise.points),-1)
pcd_noise_idx = np.array([i for i in range(len(pcd_noise.points))])

In [16]:
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

In [17]:
for noise_idx in pcd_noise_idx:
    k, neighbor_idx, _ = pcd_tree.search_hybrid_vector_3d(pcd_noise.points[noise_idx] ,0.02, 1)
    if k > 0:
        pcd_noise_label[noise_idx] = pcd_label[neighbor_idx]


/tmp/ipykernel_18208/533084165.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pcd_noise_label[noise_idx] = pcd_label[neighbor_idx]


In [10]:
pcd_noise_1 = pcd_noise.select_by_index(pcd_noise_idx[pcd_noise_label!= -1])
pcd_noise_2 = pcd_noise.select_by_index(pcd_noise_idx[pcd_noise_label== -1])
pcd_noise_2.colors = o3d.utility.Vector3dVector(
    np.tile(np.array([0,1,0]), (len(pcd_noise_2.colors), 1))
    )
pcd_noise_1.colors = o3d.utility.Vector3dVector(
    np.tile(np.array([0,0,1]), (len(pcd_noise_1.colors), 1))
    )
o3d.visualization.draw_geometries([
    pcd, pcd_noise_1, pcd_noise_2
    ])

In [18]:
for noise_idx in pcd_noise_idx[pcd_noise_label!= -1]:
    pcd.points.extend(o3d.utility.Vector3dVector(pcd_noise.points[noise_idx].reshape((-1,3))))
    pcd.colors.extend(o3d.utility.Vector3dVector(pcd_noise.colors[noise_idx].reshape((-1,3))))
    # pcd.colors.extend(o3d.utility.Vector3dVector(np.array([[0,0,1]])))
    pcd_idx = np.append(pcd_idx, len(pcd_idx))
    pcd_label= np.append(pcd_label , pcd_noise_label[noise_idx])

In [46]:
o3d.visualization.draw_geometries([
    pcd
    ])